<a href="https://colab.research.google.com/github/merveoktay/Sentiment-Analysis-With-Natural-Language-Processing/blob/main/DuyguAnalizi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from random import shuffle

import gensim
from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

import numpy

from sklearn.linear_model import LogisticRegression

In [ ]:
class TaggedLineSentence(object):
    def __init__(self, sources):
        self.sources = sources

        flipped = {}

        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')

    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    yield TaggedDocument(words=utils.to_unicode(line).split(), tags=[prefix + '_%s' % item_no])

    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    self.sentences.append(TaggedDocument(words=utils.to_unicode(line).split(), tags=[prefix + '_%s' % item_no]))
        return self.sentences

    def shuffle_sentences(self):
        return shuffle(self.sentences)

In [ ]:
sources = {'test-neg.txt':'TEST_NEG', 'test-pos.txt':'TEST_POS', 'train-neg.txt':'TRAIN_NEG', 'train-pos.txt':'TRAIN_POS', 'train-unsup.txt':'TRAIN_UNS'}

sentences = TaggedLineSentence(sources)

In [ ]:
model = gensim.models.Doc2Vec(min_count=1, window=10, vector_size=100, sample=1e-4, negative=5, workers=7,epochs=10)

In [ ]:
model.build_vocab(sentences.to_array())

In [ ]:

for epoch in range(10):
    sentences.shuffle_sentences()
    model.train(sentences.sentences,total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
model.most_similar('bad')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('horrible', 0.768563985824585),
 ('terrible', 0.7664307951927185),
 ('awful', 0.7474057674407959),
 ('lousy', 0.7349801063537598),
 ('good', 0.7325852513313293),
 ('poor', 0.6781084537506104),
 ('crappy', 0.6780505180358887),
 ('stupid', 0.6642593145370483),
 ('lame', 0.6439577341079712),
 ('abysmal', 0.6265741586685181)]

In [ ]:
model.docvecs['TEST_POS_1']

array([-0.81932825,  1.1658907 , -0.131721  , -0.99137104,  1.0851194 ,
       -0.52426493, -1.655419  ,  3.3599055 , -0.5064539 ,  4.6183505 ,
        1.4380053 , -0.18707235,  1.2179976 ,  3.1181254 , -2.1045868 ,
       -0.03082474,  1.2459667 ,  2.0850515 ,  2.827038  , -2.566438  ,
       -3.5046918 , -2.5224845 , -2.3603911 , -2.6857278 , -1.603105  ,
       -0.84064955, -1.8690617 ,  1.7854699 , -2.2160635 ,  3.647716  ,
       -2.5879774 , -1.5349429 ,  1.3413051 ,  1.1177057 , -2.1046093 ,
        3.583703  ,  0.13615225,  0.22421911, -0.31142008,  0.5751923 ,
       -0.8677877 , -1.2149892 , -3.2864916 ,  0.62466496,  0.03546249,
       -2.2007332 ,  0.75940526,  1.8153694 , -0.2088404 , -1.4928386 ,
       -1.9353749 ,  2.8752623 , -0.37913182,  1.4065211 ,  0.25453573,
       -0.91164446,  3.1279285 ,  2.6503124 , -0.5448196 ,  0.8666329 ,
        2.0531476 ,  0.73882806,  2.4862692 , -1.314558  , -2.3144603 ,
       -1.8327188 , -3.9278417 , -1.0618027 ,  0.59809345,  1.88

In [ ]:

model.save('./imdb.d2v')

In [ ]:
model = Doc2Vec.load('./imdb.d2v')

In [ ]:
train_arrays = numpy.zeros((25000, 100))
train_labels = numpy.zeros(25000)

for i in range(12500):
    prefix_train_pos = 'TRAIN_POS_' + str(i)
    prefix_train_neg = 'TRAIN_NEG_' + str(i)
    train_arrays[i] = model.docvecs[prefix_train_pos]
    train_arrays[12500 + i] = model.docvecs[prefix_train_neg]
    train_labels[i] = 1
    train_labels[12500 + i] = 0

In [ ]:
print (train_arrays)

[[-1.4937489   0.97814322  0.13480209 ...  0.67164838  0.81943232
  -0.31070155]
 [ 1.62246215 -1.70256221  4.19053555 ...  3.84320211 -3.77523804
   2.47806525]
 [ 2.66562986  0.6054129   1.46769524 ... -1.27195287  1.16642034
   1.80115032]
 ...
 [-3.318156    0.33329409 -1.05856085 ... -1.36672914 -0.34020638
  -0.01758691]
 [-1.29420424 -0.93739474  0.32219309 ... -0.50540739  1.56871355
   3.682446  ]
 [ 0.08774284 -0.68969268  0.58432537 ... -0.72339606 -0.47101033
  -0.4646793 ]]


In [ ]:
print (train_labels)

[1. 1. 1. ... 0. 0. 0.]


In [ ]:
test_arrays = numpy.zeros((25000, 100))
test_labels = numpy.zeros(25000)

for i in range(12500):
    prefix_test_pos = 'TEST_POS_' + str(i)
    prefix_test_neg = 'TEST_NEG_' + str(i)
    test_arrays[i] = model.docvecs[prefix_test_pos]
    test_arrays[12500 + i] = model.docvecs[prefix_test_neg]
    test_labels[i] = 1
    test_labels[12500 + i] = 0

In [ ]:
classifier = LogisticRegression()
classifier.fit(train_arrays, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
classifier.score(test_arrays, test_labels)

0.86972